# Calculate validation points for testing other models

We essentially do this by generating 100 LHS points with a different fixed seed.

In [1]:
# Import computational environment

import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize

#Our expensive physics simulation
import run_simple_energy_loss

import subprocess
import pandas as pd

from joblib import Parallel, delayed

In [2]:
# Define simulation
simulation=run_simple_energy_loss.run_simulation

In [3]:
# Define ranges

pT_range = (1, 10, 10) # Compute more observable than needed to avoid boundary unstabilities.
pT_obs = [3, 5, 7] # p_T Observables we will use for the analysis
prior_ranges= {'alpha_s':(0.1,0.4), 'exponent_inel':(-2,2), 'exponent_el':(-2,2), 'scale_inel':(0.1, 0.4), 'scale_el':(0.1, 0.4)}
true_values= {'alpha_s': 0.2, 'exponent_inel': -1, 'exponent_el': 1, 'scale_inel': 0.3, 'scale_el': 0.3}

In [4]:
def generate_lhs(npoints, ndim, seed):
    """
    Generate a maximin Latin-hypercube sample (LHS) with the given number of
    points, dimensions, and random seed.
    
    Reproduced from js-sims-bayes
    """
    print(
        'generating maximin LHS: '
        'npoints = %d, ndim = %d, seed = %d'\
        % (npoints, ndim, seed)
    )

    print('generating using R')
    proc = subprocess.run(
        ['R', '--slave'],
        input="""
        library('lhs')
        set.seed({})
        write.table(maximinLHS({}, {}), col.names=FALSE, row.names=FALSE)
        """.format(seed, npoints, ndim).encode(),
        stdout=subprocess.PIPE,
        check=True
    )

    lhs = np.array(
        [l.split() for l in proc.stdout.splitlines()],
        dtype=float
    )

    return lhs

In [5]:
# Generate validation points

ndim = 5
npoints = 100
validation_lhspoints = generate_lhs(npoints,ndim,123456789)
validation_points = np.zeros(validation_lhspoints.shape)

validation_points[:,0] = validation_lhspoints[:,0]*(prior_ranges['alpha_s'][1]-prior_ranges['alpha_s'][0])+prior_ranges['alpha_s'][0]
validation_points[:,1] = validation_lhspoints[:,1]*(prior_ranges['exponent_inel'][1]-prior_ranges['exponent_inel'][0])+prior_ranges['exponent_inel'][0]
validation_points[:,2] = validation_lhspoints[:,2]*(prior_ranges['exponent_el'][1]-prior_ranges['exponent_el'][0])+prior_ranges['exponent_el'][0]
validation_points[:,3] = validation_lhspoints[:,3]*(prior_ranges['scale_inel'][1]-prior_ranges['scale_inel'][0])+prior_ranges['scale_inel'][0]
validation_points[:,4] = validation_lhspoints[:,4]*(prior_ranges['scale_el'][1]-prior_ranges['scale_el'][0])+prior_ranges['scale_el'][0]


generating maximin LHS: npoints = 100, ndim = 5, seed = 123456789
generating using R


PermissionError: [Errno 13] Permission denied: 'R'

In [7]:
# Evaluate simulation at validation points

cores = 4
validation_runs=Parallel(n_jobs=cores)(delayed(simulation)(point.reshape(1,-1), *pT_range) for point in validation_points)

In [33]:
validation_runs = np.array(validation_runs).reshape(100,10)

Note: Because the likelihood function depends on the data and nominally this can be changed in the notebooks, I do not calculate the likelihood at the validation points. However, this is simple enough to do later.

In [32]:
param_keys = ['alpha_s','exponent_inel','exponent_el','scale_inel', 'scale_el']
result_keys = ['pT {} GeV'.format(str(i)) for i in np.linspace(*pT_range)]

In [29]:
# Compile validation results into dataframe

validation_df = pd.DataFrame(validation_points,columns=param_keys)
for i in range(len(result_keys)):
    validation_df[result_keys[i]] = validation_runs[:,i]

In [30]:
validation_df

,alpha_s,exponent_inel,exponent_el,scale_inel,scale_el,pT 1.0 GeV,pT 2.0 GeV,pT 3.0 GeV,pT 4.0 GeV,pT 5.0 GeV,pT 6.0 GeV,pT 7.0 GeV,pT 8.0 GeV,pT 9.0 GeV,pT 10.0 GeV
0,0.109821,1.850299,0.034996,0.336278,0.380376,0.950795,0.962387,0.864056,0.827264,0.809096,0.799168,0.793516,0.790076,0.786762,0.781527
1,0.399103,-0.197858,1.127105,0.249366,0.339815,0.635271,0.774098,0.082021,0.028302,0.014473,0.009201,0.006678,0.005109,0.003796,0.002599
2,0.455339,2.185814,-0.427819,0.234760,0.220727,0.398766,0.485631,0.049245,0.016747,0.008783,0.005852,0.004475,0.003616,0.002865,0.002117
3,0.288840,-1.150082,-2.107703,0.185754,0.148988,0.763174,0.851604,0.269264,0.164953,0.126275,0.107754,0.097629,0.091210,0.085356,0.077922
4,0.315238,-0.419635,-1.349251,0.204296,0.389159,0.430821,0.489159,0.117428,0.058139,0.037375,0.028015,0.023223,0.020249,0.017293,0.013277
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.201795,2.701999,-2.940034,0.104412,0.126623,0.960522,1.118351,0.221370,0.110299,0.075136,0.059598,0.051375,0.046182,0.041721,0.036760
96,0.367011,-2.370173,-2.728106,0.135542,0.437961,0.013723,0.015640,0.002381,0.000715,0.000288,0.000177,0.000164,0.000153,0.000111,0.000023
97,0.370041,-1.517319,-2.651501,0.410857,0.107980,0.814085,1.178614,0.016412,0.002557,0.000892,0.000463,0.000297,0.000208,0.000147,0.000099
98,0.063957,-2.904640,0.657719,0.155727,0.102910,0.978844,0.984091,0.938638,0.920894,0.911962,0.907026,0.904197,0.902475,0.900821,0.898193


In [31]:
# Write dataframe to file
validation_df.to_csv('validation.csv')